# pyISC Example: Anomaly Detection with Classes
In this example, we extend the multivariate example to the use of classes. ISC also makes it possible to compute the anomaly score for different classes, so that apples are compared to apples an d not to oranges. In addition, it is also possibel to the anomaly detector to classify unknwon examples.

In [1]:
import pyisc;
import numpy as np
from scipy.stats import poisson, norm, multivariate_normal
%matplotlib inline
from pylab import plot, figure

## Data with Classification
Create a data set with 3 columns from different probablity distributions.

In [2]:
n_classes = 3
normal_len = 10000
anomaly_len = 15
data = None
for i in range(n_classes):
    po_normal = poisson(10+i)
    po_normal2 = poisson(2+i)
    gs_normal = norm(1+i, 12)
    tmp = np.column_stack(
        [
            [1] * (normal_len),
            list(po_normal.rvs(normal_len)),
            list(po_normal2.rvs(normal_len)),
            list(gs_normal.rvs(normal_len)),
            [i] * (normal_len),
        ]
    )
    if data is None:
        data = tmp
    else:
        data = np.r_[data,tmp]
# Add anomalies
for i in range(n_classes):
    po_anomaly = poisson(25+i)
    po_anomaly2 = poisson(3+i)
    gs_anomaly = norm(2+i,30)

    tmp = np.column_stack(
        [
            [1] * (anomaly_len),
            list(po_anomaly.rvs(anomaly_len)),
            list(po_anomaly2.rvs(anomaly_len)),
            list(gs_anomaly.rvs(anomaly_len)),
            [i] * (anomaly_len),
        ]
    )
    if data is None:
        data = tmp
    else:
        data = np.r_[data,tmp]

## Anomaly Detector
Create an anomaly detector using as first argument the used statistical models. The we use  
- a onesided Poisson distribution for modelling the first fequency column (column 1) (as in the first example),
- a twosided Poisson distribution for the second frequency column (column 2),
- and a Gaussin (Normal) distribution for the last column (column 3).

Given that we now have more than one variable, it is necessary to also add a method to combine the output from the statistical models, which in this case is the maximum anomaly score of each component model:

In [3]:
anomaly_detector = pyisc.AnomalyDetector(
    component_models=[
        pyisc.P_PoissonOnesided(1,0), # columns 1 and 0
        pyisc.P_Poisson(2,0), # columns 2 and 0
        pyisc.P_Gaussian(3) # column 3
    ],
    output_combination_rule=pyisc.cr_max
)

Train the anomaly detector

In [4]:
anomaly_detector.fit(data, y=4); # y is the class column or an array with classes

Compute the anomaly scores for each data point

In [5]:
scores = anomaly_detector.anomaly_score(data, y=4)

## Anomaly Scores with Classes
Now we can print some example of normal frequencies vs. anomaly scores for the 15 first normal data points:

In [6]:
from pandas import DataFrame
df= DataFrame(data[:15], columns=['Class','#Days', 'Freq1','Freq2','Measure'])
df['Anomaly Score'] = scores[:15]
print df.to_string()

    Class  #Days  Freq1      Freq2  Measure  Anomaly Score
0     1.0   10.0    1.0  -5.323240      0.0       0.503882
1     1.0   12.0    3.0 -27.405024      0.0       3.953366
2     1.0    9.0    1.0 -10.202590      0.0       1.030416
3     1.0    5.0    4.0   1.821344      0.0       1.935028
4     1.0    9.0    2.0 -13.462644      0.0       1.453234
5     1.0   13.0    4.0 -12.110243      0.0       1.935028
6     1.0    9.0    4.0  -3.968939      0.0       1.935028
7     1.0    8.0    2.0   5.743880      0.0       0.367626
8     1.0   16.0    3.0   7.811876      0.0       2.976844
9     1.0   12.0    3.0  21.482652      0.0       2.407911
10    1.0    7.0    2.0   8.727730      0.0       0.652329
11    1.0    9.0    1.0  14.644586      0.0       1.354108
12    1.0   12.0    4.0 -14.153176      0.0       1.935028
13    1.0   11.0    0.0   2.944639      0.0       1.277888
14    1.0    9.0    3.0  16.300750      0.0       1.584821


The anomalous frequencies vs. anomaly scores for the 15 anomalous data points:

In [7]:
df= DataFrame(data[-45:], columns=['#Days', 'Freq1','Freq2','Measure','Class'])
df['Anomaly Score'] = scores[-45:]
print df.to_string()


    #Days  Freq1  Freq2    Measure  Class  Anomaly Score
0     1.0   18.0    2.0   5.685578    0.0       4.193214
1     1.0   25.0    4.0 -27.246648    0.0       9.868429
2     1.0   30.0    1.0   1.020683    0.0      15.068624
3     1.0   21.0    3.0  11.085123    0.0       6.371499
4     1.0   27.0    2.0 -26.080290    0.0      11.845223
5     1.0   28.0    2.0  -8.480246    0.0      12.886355
6     1.0   25.0    5.0 -17.730669    0.0       9.868429
7     1.0   18.0    3.0   2.526869    0.0       4.193214
8     1.0   25.0    1.0  23.733959    0.0       9.868429
9     1.0   23.0    3.0  93.113652    0.0       8.040305
10    1.0   21.0    2.0 -14.640704    0.0       6.371499
11    1.0   22.0    7.0   6.076633    0.0       7.185276
12    1.0   33.0    4.0 -16.653146    0.0      18.577359
13    1.0   27.0    5.0 -29.865255    0.0      11.845223
14    1.0   30.0    5.0  38.370337    0.0      15.068624
15    1.0   32.0    3.0  20.775388    1.0      15.396908
16    1.0   30.0    3.0 -28.270

As can be seen above, the anomalous data also have higher anomaly scores than the normal frequencies as it should be.</b><br/><br/>
This becomes even more visible if we plot the anomaly scores (y-axis) against each data point (x-axis): 

In [8]:
plot(scores, '.');

We can also look at the details of each column in terms of their individual anomaly scores:

In [9]:
score_details = anomaly_detector.anomaly_score_details(data,y=4)

In [10]:
df= DataFrame(data[-45:], columns=['#Days', 'Freq1','Freq2','Measure','Class'])
df['Anomaly:Freq1'] = [detail[2][0] for detail in score_details[-45:]]   # Anomaly Score of Freq1
df['Anomaly:Freq2'] = [detail[2][1] for detail in score_details[-45:]]   # Anomaly Score of Freq2
df['Anomaly:Measure'] = [detail[2][2] for detail in score_details[-45:]] # Anomaly Score of Measure
df['Anomaly Score'] = [detail[0] for detail in score_details[-45:]]      # Combined Anomaly Score
df

,#Days,Freq1,Freq2,Measure,Class,Anomaly:Freq1,Anomaly:Freq2,Anomaly:Measure,Anomaly Score
0,1.0,18.0,2.0,5.685578,0.0,4.193214,0.050020,0.362503,4.193214
1,1.0,25.0,4.0,-27.246648,0.0,9.868429,1.935028,3.918412,9.868429
2,1.0,30.0,1.0,1.020683,0.0,15.068624,0.250994,0.004434,15.068624
3,1.0,21.0,3.0,11.085123,0.0,6.371499,1.122022,0.909487,6.371499
4,1.0,27.0,2.0,-26.080290,0.0,11.845223,0.050020,3.665745,11.845223
5,1.0,28.0,2.0,-8.480246,0.0,12.886355,0.050020,0.830331,12.886355
6,1.0,25.0,5.0,-17.730669,0.0,9.868429,2.929363,2.097029,9.868429
7,1.0,18.0,3.0,2.526869,0.0,4.193214,1.122022,0.109131,4.193214
8,1.0,25.0,1.0,23.733959,0.0,9.868429,0.250994,2.814497,9.868429
9,1.0,23.0,3.0,93.113652,0.0,8.040305,1.122022,NaN,8.040305


Above, the last column corresponds to the same anomaly score as before, where we can se that it corresponds to the maximum of the individual anomaly score to the left, thus, it is the result of the combination rule specified to the anomaly detector.

## Anomaly Detector as Classifier
Let us create a data set with unkown classes from the same distributions as above:

In [ ]:
data2 = None
true_classes = []
length = 1000
for i in range(n_classes):
    po_normal = poisson(10+i)
    po_normal2 = poisson(2+i)
    gs_normal = norm(1+i, 12)
    tmp = np.column_stack(
        [
            [1] * (length),
            list(po_normal.rvs(length)),
            list(po_normal2.rvs(length)),
            list(gs_normal.rvs(length)),
            [-1.0] * (length),
        ]
    )
    
    true_classes += [i] * length
    
    if data2 is None:
        data2 = tmp
    else:
        data2 = np.r_[data2,tmp]

Then, we can also use the anomaly detector as a classifier to predict the class for each instance as below:

In [ ]:
from pandas import DataFrame
from sklearn.metrics import accuracy_score
result = DataFrame(columns=['Algorithm','Accuracy'])
clf = pyisc.SklearnClassifier.clf(anomaly_detector)
predicted_classes = clf.predict(data2)
acc = accuracy_score(true_classes, predicted_classes)
result.loc[0] = ['pyISC classifier', acc]

We can also compare it to some available classifiers in Scikit-learn (http://scikit-learn.org/):

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

X = data.T[:-1].T
y = data.T[-1]
count = 1
for name, clf in zip(['GaussianNB',
                      'KNeighborsClassifier', 
                      'RandomForestClassifier'],
                     [GaussianNB(), 
                      KNeighborsClassifier(n_neighbors=1000,weights='distance'), 
                      RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)]):
    clf.fit(X,y);

    predicted_classes_SK= clf.predict(data2.T[:-1].T)
    acc = accuracy_score(true_classes,predicted_classes_SK)
    result.loc[count] = [name, acc]
    count += 1

result

The pyISC classifier performce better, most likely due to being closer to using the true distributions.